In [2]:
import pandas as pd
import os

file_path = os.path.normpath(os.getcwd() + os.sep + os.pardir) + "\\cutfree-simulations\\runtime_data.csv"

In [3]:
df = pd.read_csv(file_path)

df.iloc[0]

Oligo                        NNNNNN
OligoLength                       6
Sites                   Any["YATR"]
TotalSites                        1
SiteLength                        4
CutFree_Randomer             NBNVVN
CutFree_Degeneracy          7.45472
CutFree_Time               0.006211
CutFreeRL_Randomer           NNVVVN
CutFreeRL_Degeneracy        7.45472
CutFreeRL_Time             0.228063
Name: 0, dtype: object

In [4]:
df.columns

Index(['Oligo', 'OligoLength', 'Sites', 'TotalSites', 'SiteLength',
       'CutFree_Randomer', 'CutFree_Degeneracy', 'CutFree_Time',
       'CutFreeRL_Randomer', 'CutFreeRL_Degeneracy', 'CutFreeRL_Time'],
      dtype='object')

In [5]:
df.loc[(df["OligoLength"] == 6) & (df["TotalSites"] == 1) & (df["SiteLength"] == 4)]

,Oligo,OligoLength,Sites,TotalSites,SiteLength,CutFree_Randomer,CutFree_Degeneracy,CutFree_Time,CutFreeRL_Randomer,CutFreeRL_Degeneracy,CutFreeRL_Time
0,NNNNNN,6,"Any[""YATR""]",1,4,NBNVVN,7.45472,0.006211,NNVVVN,7.45472,0.228063


In [30]:
inputs = df[["OligoLength", "TotalSites", "SiteLength"]]
cutfree_ouputs = df[["CutFree_Time", "CutFree_Degeneracy", "CutFreeRL_Time", "CutFreeRL_Degeneracy"]]

In [31]:
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs, cutfree_ouputs, test_size=0.2, random_state=42)
cutfree_model = linear_model.LinearRegression()
cutfree_model.fit(X_train, y_train)

y_pred = cutfree_model.predict(X_test)
mse = np.mean((y_pred - y_test)**2)
print(y_pred[:10])
print(y_test[:10])
print(mse/len(y_test))

[[18.07458947 36.18372896  2.0345258  34.9217892 ]
 [ 2.75656461 25.66388572  0.60326939 24.63023059]
 [ 6.72299378 14.60671906  1.21957953 13.72777469]
 [ 8.42253787 23.00223337  0.6795082  20.84963758]
 [18.37852023 42.05773394  1.93571167 40.65112698]
 [16.52198592 40.20267773  1.2812093  37.85328048]
 [-2.59984195 17.61987049  0.06268803 16.49870418]
 [-9.94809517 14.03162857 -0.34852541 13.59808191]
 [ 5.78627274 38.5486998   1.44062097 38.79103723]
 [25.42284269 39.77197088  2.44573924 37.82241147]]
      CutFree_Time  CutFree_Degeneracy  CutFreeRL_Time  CutFreeRL_Degeneracy
1246     53.568349           35.873754        2.174671             34.265445
862       0.092182           27.268306        0.744672             26.523043
298       0.175804           13.693045        0.609379             12.594432
855       0.199398           24.495717        0.736367             22.076478
1534     20.187803           43.040792        2.174223             41.602382
1603      0.375691         